In [0]:
import os
import keras
import itertools

os.environ['CUDA_VISIBLE_DEVICES']='0'

In [0]:
train,test=keras.datasets.cifar10.load_data()

In [0]:
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y   

In [0]:
x_train, y_train = train
x_test, y_test = test

x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [0]:
from keras.layers import BatchNormalization

def build_mlp(input_shape,output_units=10,num_neurons=[512,256,128]):
  input_layer=keras.layers.Input(input_shape)

  for i,n_units in enumerate(num_neurons):
    if i==0:
      x=keras.layers.Dense(units=n_units,activation='relu',name='hidden_layer'+str(i+1))(input_layer)
      x=BatchNormalization()(x)
    else:
      x=keras.layers.Dense(units=n_units,activation='relu',name='hidden_layer'+str(i+1))(x)
      x=BatchNormalization()(x)
  out=keras.layers.Dense(units=output_units,activaion='softmax',name='output')(x)

  model=keras.models.Model(inputs=[input_layer],outputs=[out])
  return model

In [0]:
LEARNING_RATE = 1e-3 #[1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
EPOCHS = 50
BATCH_SIZE = 256
OPTIMIZER = [keras.optimizers.SGD, keras.optimizers.RMSprop, keras.optimizers.Adagrad, keras.optimizers.Adam]

In [0]:
from keras.callbacks import ReduceROnPlateau

ft=[0.5,0.2,0.7,0.9]
pe=[5,3,7,9]
reduce_lr=ReduceLROnPlateau(factor=ft,min_lr=1e-12,monitor='val_loss',patience=pe,verbose=1)

In [0]:
results = {}
for fct,pte, opti in itertools.product(ft,pe, OPTIMIZER):
    keras.backend.clear_session() 
    print("Experiment with Optimizer = %s ,factor=%s , pte=%s" % (str(opti),str(fct),str(pte)))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    
    optimizer = opti
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr])
    
    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = "exp-lr-%s-optimizer-%s-factor-%s-patience-%s" % (str(lr), str(opti),str(fct),str(pte))
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 
    
NUM_COLORS = len(results.keys())
cm = plt.get_cmap('gist_rainbow')
color_bar = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()